In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed,
    DataCollatorForLanguageModeling,
)
import evaluate
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from huggingface_hub import login
import os

pd.set_option('display.max_colwidth', None)

In [2]:
load_dotenv()

RANDOM_SEED = int(os.getenv("RANDOM_SEED"))
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

set_seed(RANDOM_SEED)
login(token=HUGGINGFACE_TOKEN)

In [3]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 4060


In [4]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
model_name=os.getenv("HUGGINGFACE_MODEL_NAME")
original_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=True,
)
model_name

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

'google/gemma-2-2b-it'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
    use_fast=False,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
MAX_TOKENS = int(os.getenv("MAX_TOKENS"))
TEMPERATURE = float(os.getenv("TEMPERATURE"))
TOP_P = float(os.getenv("TOP_P"))
REPETITION_PENALTY = float(os.getenv("REPETITION_PENALTY"))

def get_chat_component(role, text):
    return {"role": role, "content": text}

def get_user_chat_component(text):
    return get_chat_component("user", text)

def get_assistant_chat_component(text):
    return get_chat_component("assistant", text)

def generate(model, prompt, 
             max_new_tokens=MAX_TOKENS, 
             temperature=TEMPERATURE, 
             top_p=TOP_P, 
             tokenizer=tokenizer, 
             repetition_penalty=REPETITION_PENALTY):
    
    """
    Generate text using the provided model and prompt
    
    Args:
        model: The language model
        prompt: Input text prompt
        max_new_tokens: Maximum length of generated text
        tokenizer: Pre-instantiated tokenizer (uses global tokenizer by default)
    
    Returns:
        List containing generated text
    """
    
    # Tokenize the input
    formatted_prompt = tokenizer.apply_chat_template(
        [get_user_chat_component(prompt)],
        tokenize=False, 
        add_generation_prompt=False,
    )
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
    )
    
    # Move to same device as model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            num_return_sequences=1,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=max_new_tokens,
            use_cache=False,
            repetition_penalty=repetition_penalty,
        )
    
    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f'INPUT [{len(prompt)}]:\n{prompt}')
    print(200*'-')
    print(f'OUTPUT [{len(generated_text)}]:\n{generated_text}')
    print(200*'-')

    return generated_text

## Data Preparation

In [8]:
GENERATED_DATA_DIR = os.getenv('GENERATED_DATA_DIR')
TRAINING_PROPORTION = float(os.getenv("TRAINING_PROPORTION"))
VALIDATION_PROPORTION = float(os.getenv("VALIDATION_PROPORTION"))
TESTING_PROPORTION = float(os.getenv("TESTING_PROPORTION"))

def format_text(entry):
    entry['text'] = f'Instruct: {entry["input"]}\nOutput: {entry["output"]}'

def compile_dataset(source_dir, 
                    training_proportion=TRAINING_PROPORTION, 
                    validation_proportion=VALIDATION_PROPORTION):
    
    training_df = pd.DataFrame()
    validation_df = pd.DataFrame()
    testing_df = pd.DataFrame()

    for source_csv_dir in os.listdir(source_dir):
        source_csv_paths = os.listdir(os.path.join(source_dir, source_csv_dir))
        for csv_path in source_csv_paths:
            csv = pd.read_csv(os.path.join(source_dir, source_csv_dir, csv_path))
            csv = csv.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
            csv['text'] = csv.apply(lambda entry: tokenizer.apply_chat_template([
                get_user_chat_component(entry["input"]),
                get_assistant_chat_component(entry["output"]),
            ], tokenize=False, add_generation_prompt=False), 1)
            total_qty = len(csv)
            validation_start = int(total_qty * training_proportion)
            validation_end = validation_start + int(total_qty * validation_proportion)
            training_df = pd.concat([training_df, csv.iloc[:validation_start]])
            validation_df = pd.concat([validation_df, csv.iloc[validation_start:validation_end]])
            testing_df = pd.concat([testing_df, csv.iloc[validation_end:]])

    training_df = training_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    validation_df = validation_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    testing_df = testing_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    return training_df, validation_df, testing_df

training_data, validation_data, testing_data = compile_dataset(GENERATED_DATA_DIR)

In [9]:
training_data

,input,output,text
0,"How does `glib` contribute to secure programming practices, specifically regarding string management?","`glib` contributes to secure programming practices, specifically regarding string management, by providing capabilities that make strings resizable and bounded. This means that strings managed by `glib` can dynamically adjust their size as needed, reducing the risk of buffer overflows that occur when fixed-size buffers are exceeded. The ""bounded"" aspect implies that `glib` helps ensure string operations stay within allocated memory limits, further enhancing security by preventing out-of-bounds writes and reads.","<bos><start_of_turn>user\nHow does `glib` contribute to secure programming practices, specifically regarding string management?<end_of_turn>\n<start_of_turn>model\n`glib` contributes to secure programming practices, specifically regarding string management, by providing capabilities that make strings resizable and bounded. This means that strings managed by `glib` can dynamically adjust their size as needed, reducing the risk of buffer overflows that occur when fixed-size buffers are exceeded. The ""bounded"" aspect implies that `glib` helps ensure string operations stay within allocated memory limits, further enhancing security by preventing out-of-bounds writes and reads.<end_of_turn>\n"
1,"What role did ""mapped network drives"" play in the attacker's activities?","Mapped network drives played a role in the attacker's activities by serving as a mechanism for distributing malware. The attacker copied malicious files between machines over these mapped network drives to infect other systems. A mapped network drive is a shortcut to a shared folder or drive on a network, allowing users to access it as if it were a local drive.","<bos><start_of_turn>user\nWhat role did ""mapped network drives"" play in the attacker's activities?<end_of_turn>\n<start_of_turn>model\nMapped network drives played a role in the attacker's activities by serving as a mechanism for distributing malware. The attacker copied malicious files between machines over these mapped network drives to infect other systems. A mapped network drive is a shortcut to a shared folder or drive on a network, allowing users to access it as if it were a local drive.<end_of_turn>\n"
2,Analyse the introduction of hypervisor with respect to attack surface.,"The introduction of a hypervisor can significantly increase the attack surface. This is primarily due to the hypervisor having a large codebase, which inevitably leads to a greater number of software bugs. Furthermore, the hypervisor operates with higher privileges than the operating system (OS) kernel. If the hypervisor itself is compromised, an attacker can gain control over the entire system more easily.","<bos><start_of_turn>user\nAnalyse the introduction of hypervisor with respect to attack surface.<end_of_turn>\n<start_of_turn>model\nThe introduction of a hypervisor can significantly increase the attack surface. This is primarily due to the hypervisor having a large codebase, which inevitably leads to a greater number of software bugs. Furthermore, the hypervisor operates with higher privileges than the operating system (OS) kernel. If the hypervisor itself is compromised, an attacker can gain control over the entire system more easily.<end_of_turn>\n"
3,What are the parameters of the `foo` function and what does it compute?,"The `foo` function takes two integer parameters, `x` and `y`. It computes the sum of these two parameters and stores the result in a local variable named `z`. The function then returns the computed sum (`z`).","<bos><start_of_turn>user\nWhat are the parameters of the `foo` function and what does it compute?<end_of_turn>\n<start_of_turn>model\nThe `foo` function takes two integer parameters, `x` and `y`. It computes the sum of these two parameters and stores the result in a local variable named `z`. The function then returns the computed sum (`z`).<end_of_turn>\n"

In [10]:
validation_data

,input,output,text
0,Explain why the `gets` function is considered an extreme risk in C programming.,"The `gets` function is considered an extreme risk because it reads a line from `stdin` into a buffer without any bounds checking. This means it will write past the end of the buffer if the input is larger than the buffer's capacity, leading to a buffer overflow, which is a common security vulnerability.","<bos><start_of_turn>user\nExplain why the `gets` function is considered an extreme risk in C programming.<end_of_turn>\n<start_of_turn>model\nThe `gets` function is considered an extreme risk because it reads a line from `stdin` into a buffer without any bounds checking. This means it will write past the end of the buffer if the input is larger than the buffer's capacity, leading to a buffer overflow, which is a common security vulnerability.<end_of_turn>\n"
1,What types of systems were compromised during the attacker's lateral movement?,"During the attacker's lateral movement, many endpoints and servers were compromised.","<bos><start_of_turn>user\nWhat types of systems were compromised during the attacker's lateral movement?<end_of_turn>\n<start_of_turn>model\nDuring the attacker's lateral movement, many endpoints and servers were compromised.<end_of_turn>\n"
2,Give an example of an authentication method that relies on 'something you have'.,"An example of an authentication method that relies on ""something you have"" could be a physical security token, a smart card, or a key. These are physical objects that the legitimate user possesses to prove their identity.","<bos><start_of_turn>user\nGive an example of an authentication method that relies on 'something you have'.<end_of_turn>\n<start_of_turn>model\nAn example of an authentication method that relies on ""something you have"" could be a physical security token, a smart card, or a key. These are physical objects that the legitimate user possesses to prove their identity.<end_of_turn>\n"
3,"What does ""reverse-engineer"" mean in the context of attackers uncovering hidden details in 'Security Through Obscurity'?","In the context of attackers uncovering hidden details in 'Security Through Obscurity', ""reverse-engineer"" means to analyze a system or software to understand its design, functionality, or underlying components, often without access to original design specifications or source code. Attackers perform this to discover how a system works, identify vulnerabilities, or bypass security measures that rely on secrecy. For example, they might analyze a compiled program (binary) to reconstruct its source code or logic, thereby revealing details that were intended to be obscured.","<bos><start_of_turn>user\nWhat does ""reverse-engineer"" mean in the context of attackers uncovering hidden details in 'Security Through Obscurity'?<end_of_turn>\n<start_of_turn>model\nIn the context of attackers uncovering hidden details in 'Security Through Obscurity', ""reverse-engineer"" means to analyze a system or software to understand its design, functionality, or underlying components, often without access to original design specifications or source code. Attackers perform this to discover how a system works, identify vulnerabilities, or bypass security measures that rely on secrecy. For example, they might analyze a compiled program (binary) to reconstruct its source code or logic, thereby revealing details that were intended to be obscured.<end_of_turn>\n"
4,"Describe the behavior of an `unsigned int` when it underflows, using `0` as an example.","When an `unsigned int` is at its minimum value, `0`, and is decremented, it wraps around to `UINT_MAX`. `UINT_MAX` represents the largest possible value for an `unsigned int`, which is `4,294,967,295`. If a variable `u2` is initialized to `0` and then `u2--` is executed, the value of `u2` becomes `4,294,967,295` due to this underflow behavior.","<bos><start_of_turn>user\nDescribe the behavior of an `unsigned int` when it underflows, using `0`

In [11]:
testing_data

,input,output,text
0,Describe the delivery stage of the Cyber Kill Chain.,"The delivery stage, the third in the Cyber Kill Chain, involves delivering the weaponized bundle to the victim through various means such as email, web, or USB drives.","<bos><start_of_turn>user\nDescribe the delivery stage of the Cyber Kill Chain.<end_of_turn>\n<start_of_turn>model\nThe delivery stage, the third in the Cyber Kill Chain, involves delivering the weaponized bundle to the victim through various means such as email, web, or USB drives.<end_of_turn>\n"
1,Provide the improved length checking condition and explain why it's better.,"The improved length checking condition is: `if (len1 <= sizeof(buf) && len2 <= sizeof(buf) && (len1 + len2 + 1 <= sizeof(buf)))`. \nThis condition is better because it addresses the `integer overflow` vulnerability more comprehensively. The original check `if (len1 + len2 + 1 <= sizeof(buf))` could be bypassed if `len1 + len2 + 1` overflowed and wrapped around to a small number, even if `len1` or `len2` individually were very large. The improved condition adds two preliminary checks: \n1. `len1 <= sizeof(buf)`: Ensures that the first length itself does not exceed the buffer's capacity. \n2. `len2 <= sizeof(buf)`: Ensures that the second length itself does not exceed the buffer's capacity. \nBy validating individual lengths first, it prevents scenarios where a single extremely large input (like `UINT_MAX`) could cause an overflow in the sum, leading to a false positive in the length check and a subsequent `Buffer Overflow`. The final check `(len1 + len2 + 1 <= sizeof(buf))` then safely verifies the combined length after the individual components have been deemed reasonable.","<bos><start_of_turn>user\nProvide the improved length checking condition and explain why it's better.<end_of_turn>\n<start_of_turn>model\nThe improved length checking condition is: `if (len1 <= sizeof(buf) && len2 <= sizeof(buf) && (len1 + len2 + 1 <= sizeof(buf)))`. \nThis condition is better because it addresses the `integer overflow` vulnerability more comprehensively. The original check `if (len1 + len2 + 1 <= sizeof(buf))` could be bypassed if `len1 + len2 + 1` overflowed and wrapped around to a small number, even if `len1` or `len2` individually were very large. The improved condition adds two preliminary checks: \n1. `len1 <= sizeof(buf)`: Ensures that the first length itself does not exceed the buffer's capacity. \n2. `len2 <= sizeof(buf)`: Ensures that the second length itself does not exceed the buffer's capacity. \nBy validating individual lengths first, it prevents scenarios where a single extremely large input (like `UINT_MAX`) could cause an overflow in the sum, leading to a false positive in the length check and a subsequent `Buffer Overflow`. The final check `(len1 + len2 + 1 <= sizeof(buf))` then safely verifies the combined length after the individual components have been deemed reasonable.<end_of_turn>\n"
2,Provide an example of how a criminal might impersonate a user's biometric data.,"For example, a criminal might impersonate a user's biometric data by creating a 'copy' of a user's fingerprint. This could be achieved by re-creating the fingerprint with a wax imprint that the criminal then places on top of their own finger to deceive the biometric scanner.","<bos><start_of_turn>user\nProvide an example of how a criminal might impersonate a user's biometric data.<end_of_turn>\n<start_of_turn>model\nFor example, a criminal might impersonate a user's biometric data by creating a 'copy' of a user's fingerprint. This could be achieved by re-creating the fingerprint with a wax imprint that the criminal then places on top of their own finger to deceive the biometric scanner.<end_of_turn>\n"
3,What are 'measures' put in place to prevent further queries?,Measures put in place to prevent further queries are security actions or configurations implemented to block or restrict unauthorized access attempts to the database.,<bos><start_of_t

In [12]:
TRAINING_ENTRIES = len(training_data)
VALIDATION_ENTRIES = len(validation_data)
TESTING_ENTRIES = len(testing_data)

print(f"""Training Proportion = {TRAINING_PROPORTION}
Training Entries = {TRAINING_ENTRIES}

Validation Proportion = {VALIDATION_PROPORTION}
Validation Entries = {VALIDATION_ENTRIES}

Testing Proportion = {TESTING_PROPORTION}
Testing Entries = {TESTING_ENTRIES}""")

Training Proportion = 0.7
Training Entries = 34757

Validation Proportion = 0.2
Validation Entries = 9735

Testing Proportion = 0.1
Testing Entries = 5528


## Initial Testing

In [13]:
prompt = "How does an attacker use gadgets in Return-Oriented Programming?"
response = generate(original_model, prompt)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


INPUT [64]:
How does an attacker use gadgets in Return-Oriented Programming?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1523]:
user
How does an attacker use gadgets in Return-Oriented Programming?


**Return-oriented programming (ROP) is a technique that exploits vulnerabilities by manipulating the program's execution flow to achieve unauthorized access or control. It involves exploiting flaws in how programs execute code, often through buffer overflows and other memory manipulation techniques.** 


Here are some ways attackers can utilize ROP for malicious purposes:

* **Exploiting Buffer Overflows:**  Attackers exploit vulnerable applications with buffer overflow attacks where they write data beyond allocated space within variables. This creates unexpected behavior like overwriting critical instructions at specific addre

In [37]:
prompt = "How does TPM ensure that data can only be decrypted on the correct platform?"
response = generate(original_model, prompt)

INPUT [76]:
How does TPM ensure that data can only be decrypted on the correct platform?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1528]:
user
How does TPM ensure that data can only be decrypted on the correct platform?
 

TPM (Trusted Platform Module) is a hardware security module embedded in many computers. It's designed to provide secure storage and management of cryptographic keys, certificates, and other sensitive information.  Here’s how it helps prevent unauthorized decryption:

**1. Secure Storage:**

* **Hardware-based Security**: The TPM stores your encryption keys securely within its dedicated memory chips. This makes them resistant to physical attacks like theft or tampering with the computer itself. Unlike software solutions where key compromise could occur through malware or system vulnerabilities, TPM ensur

## Training Base Model

In [15]:
tokenized_training_data = [tokenizer(entry) for entry in training_data['text']]
tokenized_training_data

[{'input_ids': [2, 2, 106, 1645, 108, 2299, 1721, 4103, 235264, 4613, 235376, 17006, 577, 10548, 19991, 12317, 235269, 14076, 10315, 2067, 4815, 235336, 107, 108, 106, 2516, 108, 235376, 235264, 4613, 235376, 45807, 577, 10548, 19991, 12317, 235269, 14076, 10315, 2067, 4815, 235269, 731, 9151, 22466, 674, 1501, 18935, 582, 107616, 578, 37463, 235265, 1417, 3454, 674, 18935, 12461, 731, 4103, 235264, 4613, 235376, 798, 86492, 8017, 1024, 2395, 685, 4880, 235269, 18442, 573, 5685, 576, 10691, 198797, 674, 5489, 1185, 8115, 235290, 1934, 71670, 708, 40884, 235265, 714, 664, 73127, 235281, 14818, 21721, 674, 4103, 235264, 4613, 235376, 7154, 7433, 2067, 8582, 4692, 2819, 36276, 6884, 13121, 235269, 4024, 55589, 6206, 731, 32842, 921, 235290, 559, 235290, 26825, 20332, 578, 21031, 235265, 107, 108, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [16]:
tokenized_validation_data = [tokenizer(entry) for entry in validation_data['text']]
tokenized_validation_data

[{'input_ids': [2, 2, 106, 1645, 108, 74198, 3165, 573, 4103, 21793, 235376, 1411, 603, 5604, 671, 7274, 5685, 575, 585, 19991, 235265, 107, 108, 106, 2516, 108, 651, 4103, 21793, 235376, 1411, 603, 5604, 671, 7274, 5685, 1861, 665, 21031, 476, 2017, 774, 4103, 48886, 235376, 1280, 476, 10691, 2346, 1089, 29282, 18604, 235265, 1417, 3454, 665, 877, 5598, 3433, 573, 1580, 576, 573, 10691, 1013, 573, 3772, 603, 8107, 1178, 573, 10691, 235303, 235256, 7924, 235269, 8133, 577, 476, 10691, 30973, 235269, 948, 603, 476, 3818, 6206, 55632, 235265, 107, 108, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [2, 2, 106, 1645, 108, 1841, 5088, 576, 5188, 1049, 72744, 2290, 573, 71411, 235303, 235256, 21337, 8069, 235336, 107, 108, 106

In [17]:
original_model = prepare_model_for_kbit_training(original_model)

In [18]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'o_proj',
        'dense',
        'gate_proj',
        'up_proj',
        'down_proj',
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)
peft_model.print_trainable_parameters()

trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


In [19]:
TRAINING_OUTPUT_DIR = os.getenv('TRAINING_OUTPUT_DIR')
TRAINING_STEPS = int(os.getenv('TRAINING_STEPS'))

peft_training_args = TrainingArguments(
    output_dir = TRAINING_OUTPUT_DIR,
    warmup_steps=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    max_steps=TRAINING_STEPS,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_validation_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
peft_trainer.train()

## Model Evaluation

In [21]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
original_model_outputs = []

for index in testing_data.index:
    original_model_outputs.append(generate(base_model, testing_data['input'][index]))

INPUT [52]:
Describe the delivery stage of the Cyber Kill Chain.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1438]:
user
Describe the delivery stage of the Cyber Kill Chain.


The **Cyber Kill Chain** is a framework developed by Lockheed Martin that describes the stages involved in an adversary's attack on a system or network, from initial reconnaissance to post-compromise actions. 


Here’s how it works:  It outlines seven distinct phases (or steps) through which attackers typically move during their attacks. These are as follows:

1. **Reconnaissance:** The attacker gathers information about the target and its defenses before launching any further action. This can include identifying vulnerabilities, understanding security protocols, mapping out infrastructure details, etc.
2. **Weaponization:** Attackers create malicious

In [23]:
SELECTED_STEP = 925
tuned_model = PeftModel.from_pretrained(base_model, 
                                        f"{TRAINING_OUTPUT_DIR}/checkpoint-{SELECTED_STEP}",
                                        torch_dtype=torch.float16,
                                        is_trainable=False)

In [24]:
prompt = "How does an attacker use gadgets in Return-Oriented Programming?"
response = generate(tuned_model, prompt)

INPUT [64]:
How does an attacker use gadgets in Return-Oriented Programming?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1612]:
user
How does an attacker use gadgets in Return-Oriented Programming?
An attacker uses a gadget by chaining it to existing code, typically at the end of one instruction and then using its address as part of subsequent instructions. This allows them to construct complex malicious sequences without directly modifying program execution flow or control data like return addresses themselves. The key is that these chained operations are still executed within the context of legitimate assembly instructions from other parts of the original function's logic. For example, after setting up arguments for `new_syscall`, they might chain together calls to functions like `foo` (which could be vulnerable) followed

In [38]:
prompt = "How does TPM ensure that data can only be decrypted on the correct platform?"
response = generate(tuned_model, prompt)

INPUT [76]:
How does TPM ensure that data can only be decrypted on the correct platform?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1654]:
user
How does TPM ensure that data can only be decrypted on the correct platform?
TPM ensures this by being embedded within a PC and requiring it to prove its legitimacy before allowing decryption. This prevents unauthorized systems from decrypting sensitive keys or data, as they would not possess the necessary hardware verification capability. Additionally, if an attacker attempts to access these cryptographic operations without proper TCB support (like integrity measurement), their actions will likely fail due to incompatibility with trusted execution environments managed by the TPM. The document also mentions "remote attestation," which implies secure communication between platforms 

In [ ]:
tuned_model_outputs = []

for index in testing_data.index:
    tuned_model_outputs.append(generate(tuned_model, testing_data['input'][index]))

INPUT [52]:
Describe the delivery stage of the Cyber Kill Chain.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
OUTPUT [1594]:
user
Describe the delivery stage of the Cyber Kill Chain.
The Delivery stage involves delivering the weaponized bundle to the victim, which can be achieved through various methods like email, web-based methods (like using infected links or attachments), USB drives, and physical media such as CDs or DVDs. The attacker also utilizes advanced techniques like exploiting vulnerabilities in applications for remote access software installation on mail servers. This method is particularly relevant when an internet connection exists but might not have been used during earlier stages if a direct attack was possible without it. After reaching the target system via these means, further actions are initiated within its net

In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_outputs,
    references=testing_data['output'].values,
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=tuned_model_outputs,
    references=testing_data['output'].values,
    use_aggregator=True,
    use_stemmer=True,
)

print(f'Original Model:\n{original_model_results}\n')
print(f'Tuned Model:\n{peft_model_results}\n')

Original Model:
{'rouge1': np.float64(0.2193652791513081), 'rouge2': np.float64(0.062039752362333014), 'rougeL': np.float64(0.13462452318875945), 'rougeLsum': np.float64(0.16682292803842746)}

Tuned Model:
{'rouge1': np.float64(0.2868358445678033), 'rouge2': np.float64(0.11901032252881874), 'rougeL': np.float64(0.17739888977002377), 'rougeLsum': np.float64(0.17739888977002377)}



In [34]:
print("Percentage ImprovementsL\n")
improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Percentage ImprovementsL

rouge1: 6.75%
rouge2: 5.70%
rougeL: 4.28%
rougeLsum: 1.06%
